<a href="https://colab.research.google.com/github/jkauffm4/Intro-to-ML/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import torch

Mounted at /content/drive


In [2]:
def binary_map(x):
  return x.map({'yes': 1, 'no': 0})

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
file_path = '/content/drive/My Drive/Intro-to-ML/Housing.csv'
sample = pd.DataFrame(pd.read_csv(file_path))
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
sample[varlist] = sample[varlist].apply(binary_map)
np.random.seed(0)
train_set, test_set = train_test_split(sample, train_size = .8, test_size = .2, random_state = 100)
Y = train_set.pop('price')
Y_test = test_set.pop('price')
junk = train_set.pop('furnishingstatus') #Gotta remove this, its not used
junk = test_set.pop('furnishingstatus') #Same with this
blerg = train_set
train_set_all = train_set.to_numpy()
test_set_all = test_set.to_numpy()
# Theres a better way to get out the values here, but I don't know it
junk = train_set.pop('mainroad')
junk = train_set.pop('guestroom')
junk = train_set.pop('hotwaterheating')
junk = train_set.pop('basement')
junk = train_set.pop('airconditioning')
junk = train_set.pop('prefarea')
junk = test_set.pop('mainroad')
junk = test_set.pop('guestroom')
junk = test_set.pop('hotwaterheating')
junk = test_set.pop('basement')
junk = test_set.pop('airconditioning')
junk = test_set.pop('prefarea')
X_train = train_set.to_numpy()
X_test = test_set.to_numpy()
Y = Y.to_numpy()
Y_test = Y_test.to_numpy()
size = len(X_train)
x0 = np.ones((size,1))

In [4]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_x, val_x, train_y, val_y):
  for epoch in range(1, n_epochs + 1):

    pred_y = model(train_x)
    train_loss = loss_fn(pred_y, train_y)

    with torch.no_grad():
      val_pred_y = model(val_x)
      val_loss = loss_fn(val_pred_y, val_y)
      assert val_loss.requires_grad == False

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
            f" Validation loss {val_loss.item():.4f}")

In [ ]:
X_standard_test = StandardScaler().fit_transform(test_set_all)
X_standard_train = StandardScaler().fit_transform(train_set_all)
train_x = torch.tensor(X_standard_train, dtype=torch.float32)
train_y = torch.tensor(Y, dtype=torch.float32)
val_x = torch.tensor(X_standard_test, dtype=torch.float32)
val_y = torch.tensor(Y_test, dtype=torch.float32)

In [ ]:
housing_nn = torch.nn.Sequential(
    torch.nn.Linear(11,8),
    torch.nn.Tanh(),
    torch.nn.Linear(8,1)
)

In [ ]:
optimizer = torch.optim.Adam(housing_nn.parameters(), lr = 1e-1)
loss_fn = torch.nn.MSELoss()
training_loop(n_epochs = 5000,
              optimizer = optimizer,
              model = housing_nn,
              loss_fn = loss_fn,
              train_x = train_x,
              train_y = train_y,
              val_x = val_x,
              val_y = val_y)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training loss 26424480104448.0000, Validation loss 25106363449344.0000
Epoch 1000, Training loss 26420409532416.0000, Validation loss 25102349500416.0000
Epoch 1500, Training loss 26416341057536.0000, Validation loss 25098333454336.0000
Epoch 2000, Training loss 26412264194048.0000, Validation loss 25094313213952.0000
Epoch 2500, Training loss 26408193622016.0000, Validation loss 25090295070720.0000
Epoch 3000, Training loss 26404120952832.0000, Validation loss 25086281121792.0000
Epoch 3500, Training loss 26400054575104.0000, Validation loss 25082267172864.0000
Epoch 4000, Training loss 26395984003072.0000, Validation loss 25078253223936.0000
Epoch 4500, Training loss 26391911333888.0000, Validation loss 25074237177856.0000
Epoch 5000, Training loss 26387838664704.0000, Validation loss 25070219034624.0000


In [ ]:
housing_nn_extended = torch.nn.Sequential(
    torch.nn.Linear(11, 11),
    torch.nn.Tanh(),
    torch.nn.Linear(11,8),
    torch.nn.Tanh(),
    torch.nn.Linear(8,5),
    torch.nn.Tanh(),
    torch.nn.Linear(5,1)
)

In [ ]:
housing_optimizer_extended = torch.optim.Adam(housing_nn_extended.parameters(), lr = 1e-1)
loss_fn = torch.nn.MSELoss()
training_loop(n_epochs = 5000,
              optimizer = housing_optimizer_extended,
              model = housing_nn_extended,
              loss_fn = loss_fn,
              train_x = train_x,
              train_y = train_y,
              val_x = val_x,
              val_y = val_y)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training loss 26474708992000.0000, Validation loss 25155961094144.0000
Epoch 1000, Training loss 26471829602304.0000, Validation loss 25153167687680.0000
Epoch 1500, Training loss 26468948115456.0000, Validation loss 25150374281216.0000
Epoch 2000, Training loss 26466072920064.0000, Validation loss 25147587166208.0000
Epoch 2500, Training loss 26463195627520.0000, Validation loss 25144795856896.0000
Epoch 3000, Training loss 26460320432128.0000, Validation loss 25142004547584.0000
Epoch 3500, Training loss 26457443139584.0000, Validation loss 25139217432576.0000
Epoch 4000, Training loss 26454565847040.0000, Validation loss 25136428220416.0000
Epoch 4500, Training loss 26451694845952.0000, Validation loss 25133641105408.0000
Epoch 5000, Training loss 26448819650560.0000, Validation loss 25130853990400.0000


In [39]:
from sklearn.datasets import load_breast_cancer
breast = load_breast_cancer()
breast_data = breast.data
breast_labels = breast.target
labels = np.reshape(breast_labels,(569,1))
final_breast_data = np.concatenate([breast_data,labels],axis=1)
breast_dataset = pd.DataFrame(final_breast_data)
features = breast.feature_names
features_labels = np.append(features, 'label')
breast_dataset.columns = features_labels
X_breast_data = breast_dataset.iloc[0:569,0:29].values
Y_breast_data = breast_dataset.iloc[:,30].values
sscale = StandardScaler()
X_breast_standardized = sscale.fit_transform(X_breast_data)
X_breast_train, X_breast_test, Y_breast_train, Y_breast_test = train_test_split(X_breast_standardized, Y_breast_data, train_size = .8, test_size = .2, random_state = 100)
X_breast_train = torch.tensor(X_breast_train, dtype=torch.float32)
X_breast_test = torch.tensor(X_breast_test, dtype=torch.float32)
Y_breast_train = torch.tensor(Y_breast_train, dtype=torch.float32)
Y_breast_test = torch.tensor(Y_breast_test, dtype=torch.float32)

In [40]:
cancer_nn = torch.nn.Sequential(
    torch.nn.Linear(29,32),
    torch.nn.Tanh(),
    torch.nn.Linear(32,1)
)

In [41]:
cancer_optimizer = torch.optim.Adam(cancer_nn.parameters(), lr = 1e-1)
loss_fn = torch.nn.MSELoss()
training_loop(n_epochs = 5000,
              optimizer = cancer_optimizer,
              model = cancer_nn,
              loss_fn = loss_fn,
              train_x = X_breast_train,
              train_y = Y_breast_train,
              val_x = X_breast_test,
              val_y = Y_breast_test)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([455])) that is different to the input size (torch.Size([455, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([114])) that is different to the input size (torch.Size([114, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training loss 0.2300, Validation loss 0.2502
Epoch 1000, Training loss 0.2299, Validation loss 0.2503
Epoch 1500, Training loss 0.2299, Validation loss 0.2502
Epoch 2000, Training loss 0.2299, Validation loss 0.2505
Epoch 2500, Training loss 0.2299, Validation loss 0.2503
Epoch 3000, Training loss 0.2299, Validation loss 0.2503
Epoch 3500, Training loss 0.2299, Validation loss 0.2502
Epoch 4000, Training loss 0.2299, Validation loss 0.2502
Epoch 4500, Training loss 0.2375, Validation loss 0.2617
Epoch 5000, Training loss 0.2299, Validation loss 0.2502


In [ ]:
cancer_nn_extended = torch.nn.Sequential(
    torch.nn.Linear(29, 32),
    torch.nn.Tanh(),
    torch.nn.Linear(32,16),
    torch.nn.Tanh(),
    torch.nn.Linear(16,8),
    torch.nn.Tanh(),
    torch.nn.Linear(8,1)
)

In [ ]:
cancer_optimizer_extended = torch.optim.Adam(cancer_nn_extended.parameters(), lr = 1e-1)
loss_fn = torch.nn.MSELoss()
training_loop(n_epochs = 5000,
              optimizer = cancer_optimizer_extended,
              model = cancer_nn_extended,
              loss_fn = loss_fn,
              train_x = X_breast_train,
              train_y = Y_breast_train,
              val_x = X_breast_test,
              val_y = Y_breast_test)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([455])) that is different to the input size (torch.Size([455, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([114])) that is different to the input size (torch.Size([114, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training loss 0.2299, Validation loss 0.2502
Epoch 1000, Training loss 0.2299, Validation loss 0.2502
Epoch 1500, Training loss 0.2299, Validation loss 0.2502
Epoch 2000, Training loss 0.2303, Validation loss 0.2511
Epoch 2500, Training loss 0.2299, Validation loss 0.2502
Epoch 3000, Training loss 0.2299, Validation loss 0.2505
Epoch 3500, Training loss 0.2314, Validation loss 0.2511
Epoch 4000, Training loss 0.2299, Validation loss 0.2543
Epoch 4500, Training loss 0.2299, Validation loss 0.2533
Epoch 5000, Training loss 0.2299, Validation loss 0.2519


In [6]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True)
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True)

100%|██████████| 170M/170M [00:04<00:00, 41.3MB/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [7]:
from torchvision import transforms
img, label = cifar10[99]
to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
imgs = torch.stack([to_tensor(img) for img, label in cifar10], dim=3)
imgs.view(3, -1).mean(dim=1)
imgs.view(3, -1).std(dim=1)

tensor([0.2470, 0.2435, 0.2616])

In [10]:
transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
cifar10 = datasets.CIFAR10(data_path, train = True, download = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train = False, download = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

In [12]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [17]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size = 64, shuffle = True)
cifar10_model = torch.nn.Sequential(torch.nn.Linear(3072, 256), torch.nn.Tanh(), torch.nn.Linear(256, 10), torch.nn.LogSoftmax(dim = 1))
learning_rate = 1e-2
optimizer = torch.optim.Adam(cifar10_model.parameters(), lr = learning_rate)
loss_fn = torch.nn.NLLLoss()
n_epochs = 100
for epoch in range(n_epochs):
  for imgs, labels in train_loader:
    batch_size = imgs.shape[0]
    outputs = cifar10_model(imgs.view(batch_size, -1))
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward
    optimizer.step()
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.323144
Epoch: 1, Loss: 2.296948
Epoch: 2, Loss: 2.345542
Epoch: 3, Loss: 2.297603
Epoch: 4, Loss: 2.398983
Epoch: 5, Loss: 2.325710
Epoch: 6, Loss: 2.273215
Epoch: 7, Loss: 2.542344
Epoch: 8, Loss: 2.423949
Epoch: 9, Loss: 2.339843
Epoch: 10, Loss: 2.376061
Epoch: 11, Loss: 2.449663
Epoch: 12, Loss: 2.387151
Epoch: 13, Loss: 2.437116
Epoch: 14, Loss: 2.415786
Epoch: 15, Loss: 2.354473
Epoch: 16, Loss: 2.392390
Epoch: 17, Loss: 2.364003
Epoch: 18, Loss: 2.481707
Epoch: 19, Loss: 2.338970
Epoch: 20, Loss: 2.300961
Epoch: 21, Loss: 2.476236
Epoch: 22, Loss: 2.290874
Epoch: 23, Loss: 2.339228
Epoch: 24, Loss: 2.362416
Epoch: 25, Loss: 2.370901
Epoch: 26, Loss: 2.294543
Epoch: 27, Loss: 2.310260
Epoch: 28, Loss: 2.431381
Epoch: 29, Loss: 2.478729
Epoch: 30, Loss: 2.385786
Epoch: 31, Loss: 2.369170
Epoch: 32, Loss: 2.375442
Epoch: 33, Loss: 2.395299
Epoch: 34, Loss: 2.335254
Epoch: 35, Loss: 2.286980
Epoch: 36, Loss: 2.270180
Epoch: 37, Loss: 2.383752
Epoch: 38, Loss: 2.502

In [20]:
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = cifar10_model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.074020


In [23]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size = 64, shuffle = True)
cifar10_model_extended = torch.nn.Sequential(torch.nn.Linear(3072, 1024), torch.nn.Tanh(), torch.nn.Linear(1024, 256), torch.nn.Tanh(), torch.nn.Linear(256, 64), torch.nn.Tanh(), torch.nn.Linear(64,10), torch.nn.LogSoftmax(dim = 1))
learning_rate = 1e-2
optimizer = torch.optim.Adam(cifar10_model_extended.parameters(), lr = learning_rate)
loss_fn = torch.nn.NLLLoss()
n_epochs = 100
for epoch in range(n_epochs):
  for imgs, labels in train_loader:
    batch_size = imgs.shape[0]
    outputs = cifar10_model(imgs.view(imgs.shape[0], -1))
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward
    optimizer.step()
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.273928
Epoch: 1, Loss: 2.411807
Epoch: 2, Loss: 2.376977
Epoch: 3, Loss: 2.428854
Epoch: 4, Loss: 2.459967
Epoch: 5, Loss: 2.335244
Epoch: 6, Loss: 2.393790
Epoch: 7, Loss: 2.468582
Epoch: 8, Loss: 2.328747
Epoch: 9, Loss: 2.256191
Epoch: 10, Loss: 2.355838
Epoch: 11, Loss: 2.366473
Epoch: 12, Loss: 2.419211
Epoch: 13, Loss: 2.311019
Epoch: 14, Loss: 2.377895
Epoch: 15, Loss: 2.313069
Epoch: 16, Loss: 2.402389
Epoch: 17, Loss: 2.316300
Epoch: 18, Loss: 2.319998
Epoch: 19, Loss: 2.473073
Epoch: 20, Loss: 2.314267
Epoch: 21, Loss: 2.371730
Epoch: 22, Loss: 2.324185
Epoch: 23, Loss: 2.338358
Epoch: 24, Loss: 2.450111
Epoch: 25, Loss: 2.439771
Epoch: 26, Loss: 2.331348
Epoch: 27, Loss: 2.385567
Epoch: 28, Loss: 2.493401
Epoch: 29, Loss: 2.419688
Epoch: 30, Loss: 2.296521
Epoch: 31, Loss: 2.422075
Epoch: 32, Loss: 2.437207
Epoch: 33, Loss: 2.455475
Epoch: 34, Loss: 2.330341
Epoch: 35, Loss: 2.322149
Epoch: 36, Loss: 2.273469
Epoch: 37, Loss: 2.466022
Epoch: 38, Loss: 2.336

In [24]:
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = cifar10_model_extended(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.094440
